In [1]:
#load all necessary packages
from gen_ai_hub.proxy.native.openai import embeddings
from gen_ai_hub.proxy.native.openai import completions
from  hdbcli import dbapi
import pandas as pd
import hana_ml.dataframe as dataframe
import os
import json

In [2]:
with open(os.path.join(os.getcwd(), '/Users/sierra/Desktop/240905_enablement_session_prep/Assignment/Enablement-hands-on/secrets/HANACloudServiceKey.json')) as f:
    hana_env_c = json.load(f)
    port_c  = hana_env_c['port']
    user_c  = hana_env_c['user']
    url_c  = hana_env_c['url']
    pwd_c  = hana_env_c['pwd']

conn = dataframe.ConnectionContext(
    address=url_c,  
    port=443,
    user=user_c,
    password=pwd_c,
    encrypt='true'
)
# Establish a second connection to the HANA Cloud database using dbapi.connect
conn1 = dbapi.connect(
    address=url_c, 
    port=443, 
    user=user_c, 
    password=pwd_c   
)

In [3]:
prompt = "What are the reviews about Tacos?"
res = embeddings.create(input=prompt, model="text-embedding-ada-002")
query_vector = res.data[0].embedding

sql = '''SELECT TOP {k}  "FILENAME","TEXT" , TO_NVARCHAR("VECTOR") AS VECTOR_STR ,"{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) as SCORING
                  FROM "GEN_AI"."REVIEWS_TARGET_SIERRA"  
                  ORDER BY "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=10, metric="COSINE_SIMILARITY",
                                                                                        qv=query_vector, sort="DESC")
hdf = conn.sql(sql)
res = hdf.head(10).collect() 

if not res.empty:
    db_results = [(row['FILENAME'],row['TEXT'], row['SCORING']) for _, row in res.iterrows()]
    new_results = []
    for i in range(len(db_results)):
        if i < len(db_results):
            filename, text,  scoring = db_results[i]
            sentiment_prompt = f"Provide sentiment in exactly one word for the: '{text}'"
            sentiment_output = completions.create(model_name="meta--llama3-70b-instruct", prompt=sentiment_prompt, max_tokens=60)
            sentiment = sentiment_output.choices[0].text.strip()
            new_tuple =  (filename,text, scoring, sentiment)
            new_results.append(new_tuple) 
                   
df_new_results = pd.DataFrame(new_results)
print(df_new_results)

                                  0  \
0  {"doc_name": "customer283.json"}   
1  {"doc_name": "customer295.json"}   
2  {"doc_name": "customer331.json"}   
3  {"doc_name": "customer297.json"}   
4  {"doc_name": "customer350.json"}   
5  {"doc_name": "customer320.json"}   
6  {"doc_name": "customer349.json"}   
7  {"doc_name": "customer305.json"}   
8  {"doc_name": "customer258.json"}   
9  {"doc_name": "customer258.json"}   

                                                   1         2  \
0  great taco's\, nacho's\, atmosphere and margar...  0.837133   
1  Although there are countless little Mexican pl...  0.828023   
2  In short\, the food was decently good\, but co...  0.818785   
3  Found this place on Yelp. The reviews speak fo...  0.813769   
4  We stopped at Tako due to the positive reviews...  0.810201   
5  Our group of 6 decided to try something differ...  0.809492   
6  Great food for the price. \nTender steak and p...  0.807759   
7  I was here a few times before and didn'